Caixeiro viajante
=================



## Introdução



Até agora, sempre que nós aplicamos os operadores de `cruzamento` ou de `mutação` nós não nos preocupamos se o indivíduo gerado por estes processo era um `indivíduo válido`. Um indivíduo válido é aquele que representa uma solução possível e bem formulada para o problema em questão.

Por exemplo, no problema das caixas binárias, [1, 0, 0, 1] é um indivíduo válido para o caso de termos 4 caixas. Um exemplo de `indivíduo inválido` para este mesmo problema seria [1, 0, 0, a], pois um dos genes está assumindo um valor fora do domínio. Outro exemplo de indivíduo inválido poderia ser [1, 1, 0], pois é um indivíduo com apenas 3 genes, sendo que o esperado eram 4 genes.

Neste experimento nós veremos estratégias para evitar que indivíduos inválidos sejam obtidos quando usamos os operadores de cruzamento e de mutação. No notebook seguinte veremos como aplicar uma penalidade para indivíduos inválidos que forem gerados durante uma busca genética com restrições.



## Objetivo



Encontrar uma solução para o problema do caixeiro viajante. Considere que ele irá visitar 5 cidades, pode iniciar sua viagem por qualquer uma destas cidades e deve retornar à cidade de início. Durante seu trajeto, não pode visitar a mesma cidade duas vezes (única exceção é a cidade inicial).



## Descrição do problema



O problema consiste em descobrir a rota de menor distância entre $n$ pontos no plano cartesiano (ou seja, $n$ pontos com coordenadas $(x,y)$). A rota pode se iniciar em qualquer um dos pontos disponíveis e deve terminar no ponto inicial, visitando todos os demais pontos apenas uma vez. Considere que a rota entre um ponto e outro é a linha reta que liga os dois pontos.

O código abaixo é um gerador de coordenadas de cidades. Use ele caso queira.



## Importações

In [1]:
import random
from itertools import permutations

from funcoes import cria_cidades

from funcoes import populacao_inicial_cv as cria_populacao_inicial
from funcoes import funcao_objetivo_pop_cv
from funcoes import funcao_objetivo_cv
from funcoes import selecao_torneio_min as funcao_selecao # esse já temos!
from funcoes import cruzamento_ordenado as funcao_cruzamento
from funcoes import mutacao_de_troca as funcao_mutacao

## Códigos e discussão



In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 50
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05
NUM_COMBATENTES_NO_TORNEIO = 3
NUM_GERACOES= 1000

# relacionadas ao problema a ser resolvido
NUMERO_DE_CIDADES= 5
CIDADES= cria_cidades(NUMERO_DE_CIDADES)

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_cv(populacao, CIDADES)

def funcao_objetivo_individuo(individuo):
    return funcao_objetivo_cv(individuo, CIDADES)

In [4]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, CIDADES)

melhor_fitness_ja_visto = float("inf")  # é assim que escrevemos infinito em python

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
    
    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]
    
    contador = 0
    
    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2
        
        contador = contador + 2   
        
    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)            
            
    # melhor individuo já visto até agora
    fitness = funcao_objetivo_pop(populacao)
    menor_fitness = min(fitness)
    if menor_fitness < melhor_fitness_ja_visto:        
        posicao = fitness.index(menor_fitness)
        melhor_individuo_ja_visto = populacao[posicao]
        melhor_fitness_ja_visto = menor_fitness    

In [5]:
# Busca testando todas as permutações

melhor_fitness_ever = float("inf")

# testando todas as permutações possíveis
for caminho in permutations(list(CIDADES.keys())):
    distancia = funcao_objetivo_individuo(caminho)
    if distancia < melhor_fitness_ever:
        melhor_fitness_ever = distancia
        melhor_resposta_ever = caminho

In [6]:
# Checando os resultados

print()
print("Melhor individuo obtido por algoritmos genéticos:")
print(melhor_individuo_ja_visto, "com distância:", melhor_fitness_ja_visto)

print()
print("Melhor individuo obtido por busca exaustiva:")
print(melhor_resposta_ever, "com distância:", melhor_fitness_ever)


Melhor individuo obtido por algoritmos genéticos:
['Cidade 3', 'Cidade 4', 'Cidade 2', 'Cidade 0', 'Cidade 1'] com distância: 1.5099489578616982

Melhor individuo obtido por busca exaustiva:
('Cidade 0', 'Cidade 2', 'Cidade 4', 'Cidade 1', 'Cidade 3') com distância: 1.5099489578616982


## Conclusão



#### SOBRE O PROBLEMA:

No experimento 7, pela primeira vez entre todos os experimentos, não sabemos a resposta correta, pois esse é um problema NP de alta complexidade! Portanto, dessa vez, não temos critério de parada fixo. Por isso, foi necessário estabelecer o número de iterações como critério de parada.

O único jeito de sabermos qual o melhor caminho para o caixeiro é analisando TODAS as possibilidades: e é isso que o nosso código retorna quando se refere a "busca exaustiva". No entanto, obter o resultado por busca exaustiva é quase impossível para um número (C) muito grande de cidades, pois a quantidade de possibilidades é calculada por (C-1)!. Logo, só temos um valor de referência (valor correto, obtido pela busca exaustiva) para baixo valores de C, visto que não é possível checar qual a melhor resposta possível do algoritmo para altos valores de C.


#### CRUZAMENTO ORDENADO:

Nesse experimento, definimos um novo tipo de cruzamento, denominado "cruzamento cruzado. Afinal, nesse caso, o "cruzamento ponto simples" não resolve, pois no problema do caixeiro, o viajante não pode repetir cidades no seu trajeto! Por isso, estabeleceremos dois pontos de corte distintos no cruzamento, que agora podem estar nos limites. Esses pontos estabelecerão uma região. O filho 1 vai herdar os genes entre os cortes do pai e o restante da mãe (sem repetir).O filho 2 herdará os genes entre os cortes da mãe e o restante do pai (sem repetir)

#### RESULTADOS 

Utilizando a constante NUMERO_DE_CIDADES= 4 (ou seja, apenas quatro cidades), ao rodar várias vezes o código, percebe-se que o resultado de DISTÂNCIA dado pelo algoritimo genético praticamente sempre coincide com o da busca exaustiva (resposta certa). Já a ordem das cidades pode variar entre as duas respostas, desde que mantenha a mesma sequência (exemplo: a sequência de cidades 1, 2, 3,4 é equivalente, em distância total, à sequência 4, 3, 2, 1).

Entretanto, utilizando um número maior de cidades, o código da busca exaustiva para de rodar (demora muito). Isso se deve ao fato de que o número de possibilidades de caminho para o caixeiro varia de acordo com a fórmula (n-1)! sendo n o número de cidades. Logo, quando temos um problema de 15 cidades, por exemplo, o número de iterações do código que calcula a resposta correta é de 1.3076744e+12. Nessa situação, não é possível comparar a resposta dada pelo algoritmo genético com a resposta certa, por isso, é importante sempre buscar os melhores parâmetros de seleção, cruzamento e mutação, com intuito de aumentar a eficiência do algoritimo genético e de fazer com que suas respostas sejam as mais próximas do correto possível (mesmo se não soubermos qual é esse "correto").


## Playground

